# Домашнее задание № 5. Матричные разложения/Тематическое моделирование

## Задание № 1

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from string import punctuation
from razdel import tokenize as razdel_tokenize
import pickle
import warnings

morph = MorphAnalyzer()
warnings.filterwarnings("ignore")


def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)


def eval_table(x, y, pipeline, n=6):
    labels = list(set(y))
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))
    kfold = StratifiedKFold(n_splits=n, shuffle=True)
    for i, (train_index, test_index) in enumerate(kfold.split(x, y)):
        pipeline.fit(x[train_index], y[train_index])
        preds = pipeline.predict(x[test_index])
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(n)]].mean(axis=1)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(n)]].std(axis=1)
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(n)]].mean(axis=1)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(n)]].std(axis=1)
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(n)]].mean(axis=1)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(n)]].std(axis=1)
    result.loc['mean'] = result.mean()
    errors /= n
    return result, errors

In [2]:
data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)

In [6]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.6, ngram_range=(1, 2), stop_words=stopwords.words('russian'),
                             tokenizer=lambda x: x.split())
X_sparse = vectorizer.fit_transform(data['description_norm'])
svd = TruncatedSVD(500)
pipelines = {'svd-sgd': Pipeline([('vec', vectorizer), ('decomp', svd), ('clf', SGDClassifier())]),
             'svd-kneighbors': Pipeline([('vec', vectorizer), ('decomp', svd), ('clf', KNeighborsClassifier())]),
             'svd-bayes': Pipeline([('vec', vectorizer), ('decomp', svd), ('scaling', MinMaxScaler()), 
                                    ('clf', MultinomialNB())]),
             'svd-randomforest': Pipeline([('vec', vectorizer), ('decomp', svd), ('clf', RandomForestClassifier())]),
             'svd-extratrees': Pipeline([('vec', vectorizer), ('decomp', svd), ('clf', ExtraTreesClassifier())]),
             'nmf-sgd': Pipeline([('vec', vectorizer), ('decomp', NMF(100)), ('clf', SGDClassifier())]),
             'nmf-kneighbors': Pipeline([('vec', vectorizer), ('decomp', NMF(100)), ('clf', KNeighborsClassifier())]),
             'nmf-bayes': Pipeline([('vec', vectorizer), ('decomp', NMF(100)), ('scaling', MinMaxScaler()),
                                    ('clf', MultinomialNB())]),
             'nmf-randomforest': Pipeline([('vec', vectorizer), ('decomp', NMF(100)), ('clf', RandomForestClassifier())]),
             'nmf-extratrees': Pipeline([('vec', vectorizer), ('decomp', NMF(100)), ('clf', ExtraTreesClassifier())])}
for count, pipe in enumerate(pipelines):
    print(f'Running pipeline {pipe} ({count + 1} out of {len(pipelines)}). ', end='')
    start = datetime.now()
    metrics_svd, errors_svd = eval_table(data['description_norm'], data['category_name'], pipelines[pipe])
    print(f'Mean f1 for pipeline {pipe} is {metrics_svd.loc["mean"]["f1"].round(3)}. Time elapsed: {datetime.now() - start}')

Running pipeline svd-sgd (1 out of 10). Mean f1 for pipeline svd-sgd is 0.758. Time elapsed: 0:00:51.132193
Running pipeline svd-kneighbors (2 out of 10). Mean f1 for pipeline svd-kneighbors is 0.471. Time elapsed: 0:00:51.702201
Running pipeline svd-bayes (3 out of 10). Mean f1 for pipeline svd-bayes is 0.111. Time elapsed: 0:00:45.568738
Running pipeline svd-randomforest (4 out of 10). Mean f1 for pipeline svd-randomforest is 0.641. Time elapsed: 0:02:43.349817
Running pipeline svd-extratrees (5 out of 10). Mean f1 for pipeline svd-extratrees is 0.623. Time elapsed: 0:01:37.652297
Running pipeline nmf-sgd (6 out of 10). Mean f1 for pipeline nmf-sgd is 0.509. Time elapsed: 0:03:41.600246
Running pipeline nmf-kneighbors (7 out of 10). Mean f1 for pipeline nmf-kneighbors is 0.507. Time elapsed: 0:04:09.596917
Running pipeline nmf-bayes (8 out of 10). Mean f1 for pipeline nmf-bayes is 0.527. Time elapsed: 0:03:17.353888
Running pipeline nmf-randomforest (9 out of 10). Mean f1 for pipelin

Если взять за основной критерий среднее значение F1, лучшее сочетание - это SVD разложение и SGD классификатор. Кроме того, этот способ один из самых лучших по скорости.

## Задание № 2

In [1]:
import gensim
import numpy as np
import re
import string
import warnings
from gensim.models.ldamulticore import LdaMulticore
from pymorphy2 import MorphAnalyzer
from razdel import tokenize as razdel_tokenize

morph = MorphAnalyzer()
warnings.filterwarnings("ignore")
punctuation = string.punctuation + '—«»…'
num_topics = 20


def normalize(text: str) -> str:
    normalized_text = [morph.parse(word.text.strip(punctuation).lower())[0].normal_form for word in
                       razdel_tokenize(text) if word.text.strip(punctuation)]
    return ' '.join(normalized_text)


def perplexity(model: LdaMulticore, corpus: list) -> float:
    return np.exp2(-model.log_perplexity(corpus))


def coherence(model: LdaMulticore, texts: list, dictionary: gensim.corpora.Dictionary):
    topics = []
    for topic_id, topic in model.show_topics(num_topics=num_topics, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics,
                                                       texts=texts,
                                                       dictionary=dictionary, coherence='c_v')
    return coherence_model_lda.get_coherence()


def print_model_report(model, corpus, texts, dictionary):
    print('Topics:')
    print(*[re.findall('"(.+?)"', i[1]) for i in model.print_topics()], sep='\n')
    print(f'Perplexity = {perplexity(model, corpus)}')
    print(f'Coherence = {coherence(model, texts, dictionary)}')


def run_simple_lda(texts):
    simple_dictionary = gensim.corpora.Dictionary((text.split() for text in texts))
    simple_dictionary.filter_extremes(no_above=0.1, no_below=10)
    simple_dictionary.compactify()
    simple_corpus = [simple_dictionary.doc2bow(text.split()) for text in texts]
    simple_lda = gensim.models.LdaMulticore(simple_corpus, num_topics, alpha='asymmetric', id2word=simple_dictionary,
                                            passes=10)
    print_model_report(simple_lda, simple_corpus, [text.split() for text in texts], simple_dictionary)


def run_lda_with_ngrams(texts):
    texts = [text.split() for text in texts]
    phrases = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4)
    phraser = gensim.models.phrases.Phraser(phrases)
    ngrammed_texts = [text for text in phraser[texts]]
    ngrammed_dictionary = gensim.corpora.Dictionary(ngrammed_texts)
    ngrammed_dictionary.filter_extremes(no_above=0.1, no_below=10)
    ngrammed_dictionary.compactify()
    ngrammed_corpus = [ngrammed_dictionary.doc2bow(text) for text in ngrammed_texts]
    lda_with_ngrams = gensim.models.LdaMulticore(ngrammed_corpus, num_topics, alpha='asymmetric',
                                                 id2word=ngrammed_dictionary, passes=10)
    print_model_report(lda_with_ngrams, ngrammed_corpus, ngrammed_texts, ngrammed_dictionary)


def run_lda_with_tf_idf(texts):
    simple_dictionary = gensim.corpora.Dictionary((text.split() for text in texts))
    simple_dictionary.filter_extremes(no_above=0.1, no_below=10)
    simple_dictionary.compactify()
    simple_corpus = [simple_dictionary.doc2bow(text.split()) for text in texts]
    tfidf = gensim.models.TfidfModel(simple_corpus, id2word=simple_dictionary)
    tf_idf_corpus = tfidf[simple_corpus]
    simple_lda = gensim.models.LdaMulticore(tf_idf_corpus, num_topics, alpha='asymmetric', id2word=simple_dictionary,
                                            passes=10)
    print_model_report(simple_lda, tf_idf_corpus, [text.split() for text in texts], simple_dictionary)


def run_uber_lda(texts):
    texts = [text.split() for text in texts]
    phrases = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4)
    phraser = gensim.models.phrases.Phraser(phrases)
    ngrammed_texts = [text for text in phraser[texts]]
    ngrammed_dictionary = gensim.corpora.Dictionary(ngrammed_texts)
    ngrammed_dictionary.filter_extremes(no_above=0.1, no_below=10)
    ngrammed_dictionary.compactify()
    ngrammed_corpus = [ngrammed_dictionary.doc2bow(text) for text in ngrammed_texts]
    tfidf = gensim.models.TfidfModel(ngrammed_corpus, id2word=ngrammed_dictionary)
    tf_idf_corpus = tfidf[ngrammed_corpus]
    lda_with_ngrams = gensim.models.LdaMulticore(tf_idf_corpus, num_topics, alpha='asymmetric',
                                                 id2word=ngrammed_dictionary, passes=10)
    print_model_report(lda_with_ngrams, tf_idf_corpus, ngrammed_texts, ngrammed_dictionary)

In [2]:
raw_texts = open('wiki_data.txt', encoding='utf-8-sig').read().splitlines()[:1000]
normalized_texts = [normalize(text) for text in raw_texts]

In [3]:
run_simple_lda(normalized_texts)

Topics:
['г', 'день', 'регион', 'праздник', 'метод', 'век', 'компания', 'святой', 'период', 'процесс']
['животное', 'чтобы', 'друг', 'использовать', 'есть', 'если', 'образ', 'жизнь', 'можно', 'земля']
['команда', 'матч', 'сезон', 'чемпионат', 'клуб', 'выиграть', 'кубок', 'турнир', 'победа', 'чемпион']
['канада', 'корабль', 'национальный', 'программа', 'университет', 'провинция', 'космический', 'округ', 'версия', 'аэропорт']
['альбом', 'песня', 'длина', 'семейство', 'смотреть', 'мм', 'американский', 'сын', 'род', 'де']
['остров', 'россия', 'экономический', 'общество', 'метр', 'километр', 'проект', 'развитие', 'земля', 'г']
['собор', 'система', 'станция', 'монастырь', 'атлетика', 'кладбище', 'орудие', 'тяжёлый', 'боливия', 'строительство']
['доктор', 'село', 'река', 'харьковский', 'км', 'серия', '7', 'совет', 's', 'штат']
['уезд', 'губерния', 'спортсмен', 'нижегородский', 'гонка', 'волость', 'венесуэла', '2000', 'раунд', '1996']
['суд', 'газета', 'рим', 'день', 'дело', 'советский', 'дека

['команда', 'матч', 'сезон', 'чемпионат', 'клуб', 'выиграть', 'кубок', 'турнир', 'победа', 'чемпион'] - идеальная тема, все слова связаны, лишних нет

In [4]:
run_lda_with_ngrams(normalized_texts)

Topics:
['остров', 'ни_один', 'не_завоевать', 'история_но', 'страна_представлять', 'медаль_сборная', 'спортсмен', 'метр', 'медаль', 'тот_число']
['клуб', 'зимний_олимпийский', 'король', 'сын', 'сборная', 'брат', 'контракт', 'команда', 'вернуться', 'де']
['компания', 'канада', 'сша', 'ссср', 'предприятие', 'завод', 'посёлок', 'основать', 'м', 'корпус']
['игра', 'село', 'население', 'течение', 'полёт', 'эксперимент', 'корабль', 'станция', 'на_расстояние', 'по_перепись']
['департамент', 'аргентина', 'лагерь', 'национальный', 'украинский', 'против', 'суд', 'правительство', 'польский', 'ссср']
['вид', 'км', 'длина', 'посёлок', 'день', 'смотреть', 'река', 'высота', 'население', 'мм']
['матч', 'остров', 'сборная', 'зелёный', 'турнир', 'выиграть', 'семейство', 'счёт', 'ирландия', 'команда']
['команда', 'спортсмен', 'игра', 'клуб', 'финал', 'матч', 'альбом', 'занять', 'сборная', 'сезон']
['г', 'здание', 'дом', 'общество', 'церковь', 'университет', 'россия', 'кладбище', 'музей', 'построить']
['у

По числовым показателям версия с n-граммами не лучше и не хуже обычной. "На глаз" в версии с n-граммами кажется больше удачных тем. Помимо спортивной, здесь ещё получилась удачная военная:
['корпус', 'дивизия', 'армия', 'полковник', 'командующий', 'соединение', 'командир', 'наградить', 'затем', 'июль']

In [5]:
run_lda_with_tf_idf(normalized_texts)

Topics:
['остров', 'фильм', 'спортсмен', 'канада', 'соревнование', 'чемпионат', 'клуб', 'атлетика', 'зимний', 'экипаж']
['село', 'уезд', 'значение', 'посёлок', 'река', 'харьковский', 'км', 'сельский', 'нижегородский', 'код']
['департамент', 'жужевать', 'км²', 'аргентина', 'провинция', 'граничить', 'рост', 'плотность', 'муниципалитет', 'статистика']
['б', 'секретарь', 'вкп', 'цк', 'атака', 'князь', 'совет', 'армия', 'село', 'чемпионат']
['хутор', 'дубовский', 'ростовский', 'поселение', 'сельский', 'динамика', 'иметься', 'численность', 'верхний', 'изображение']
['верховный', 'экипаж', 'лагерь', 'ссср', 'финал', 'корпус', 'бронзовый', 'король', 'заезд', 'село']
['существо', 'остров', 'живой', 'иран', 'уезд', 'компания', 'концепция', 'роман', 'оскар', 'полёт']
['сезон', 'лагерь', 'завод', 'тур', 'музей', 'совместно', 'парк', 'н', '№', 'кубок']
['фотография', 'квартира', 'ночь', 'украина', 'уезд', 'фильм', 'старое', 'павел', 'подняться', 'нижегородский']
['фамилия', 'испанский', 'итальянски

По числовым показателям версия с TF-IDF уступает обоим предыдущим версиям. В целом, темы угадываются, но почти в каждой есть лишние слова. Самая удачная тема, пожалуй, музыкальная: ['альбом', 'песня', 'in', 'композитор', 'музыка', 'фильм', 'the', 'музыкальный', 'пилот', 'сказать']

In [6]:
run_uber_lda(normalized_texts)

Topics:
['остров', 'фильм', 'уезд', 'значение', 'игра', 'канада', 'клуб', 'вид', 'г', 'спортсмен']
['не_завоевать', 'история_но', 'ни_один', 'страна_представлять', 'медаль_сборная', 'зимний_олимпийский', 'медаль', 'венесуэла', 'игра_1992', 'женщина_принимать']
['фамилия', 'метр_над', 'известный', 'испанский', 'итальянский', 'диаметр', 'семя', 'мексика', 'свободный', 'лист']
['село', 'харьковский_область', 'по_перепись', 'посёлок', 'на_расстояние', 'сельский_совет', 'м_ж', '2001_год', 'код_коатуа', 'население']
['пункт', 'железнодорожный', 'цветок', 'украина', 'сборник', 'литература', 'миссия', 'николай', 'село', 'км_к']
['сфера', 'ты', 'сезон', 'доктор', 'спортсмен', 'игра', 'место_занятой', 'таблица', 'воздух', 'разработать']
['региональный', 'регион', 'национальный', 'код_коатуа', 'автомобильный_дорога', 'м_ж', 'по_перепись', 'посёлок', 'железный_дорога', 'харьковский_область']
['автомобиль', 'суд', 'круг', 'трасса', '1965_год', 'страна', 'альбом', 'фраза', 'финансовый', 'европейский

Числовые показатели почти такие же, как и в обычной TF-IDF версии. На глаз получается тоже похоже на TF-IDF. Самые удачные темы - спортивные, например: ['не_завоевать', 'история_но', 'ни_один', 'страна_представлять', 'медаль_сборная', 'зимний_олимпийский', 'медаль', 'венесуэла', 'игра_1992', 'женщина_принимать']

#### В общем и целом, наиболее удачной, на мой взгляд, получилась версия с n-граммами без TF-IDF.